# You Do 2

Importing libraries and reading movielens dataset

In [1]:
import numpy as np
import pandas as pd

df = pd.read_csv('https://files.grouplens.org/datasets/movielens/ml-100k/u.data', delimiter=r'\t',
names=['user_id', 'item_id', 'rating', 'timestamp'])
R = df.pivot(index='user_id', columns='item_id', values='rating').values

<ipython-input-1-57d858efa56c>:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('https://files.grouplens.org/datasets/movielens/ml-100k/u.data', delimiter=r'\t',


In [2]:
R

array([[ 5.,  3.,  4., ..., nan, nan, nan],
       [ 4., nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [ 5., nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan,  5., nan, ..., nan, nan, nan]])

In [3]:
# I need indexes of non-zero elements to train and score on.
irow, jcol = np.where(~np.isnan(R))

Train, validation and test split. I used validation set to optimize lambda hyperparameter

In [4]:
def split_data(R,irow,jcol):
    idx_all = np.random.choice(np.arange(100_000), 2000, replace=False)
    idx = np.random.choice(np.arange(2000), 1000, replace=False)
    val_idx = [val_id for val_id in idx_all if val_id not in idx]
    test_irow = irow[idx]
    test_jcol = jcol[idx]
    val_irow = irow[val_idx]
    val_jcol = jcol[val_idx]
    R_copy = R.copy()
    R_copy[test_irow, test_jcol] = np.nan
    R_copy[val_irow, val_jcol] = np.nan
    R_test_mask = R.copy() # Will be used only dor prediction
    R_test_mask[test_irow, test_jcol] = np.nan
    return R_copy, test_irow, test_jcol, val_irow, val_jcol, R_test_mask

In [5]:
# Loss function
def loss(b_user, b_item, R, irow, jcol):
    loss = 0
    for i, j in zip(irow, jcol):
        if np.isnan(R[i, j]):
            continue
        loss += (R[i, j] - b_user[i] - b_item[j]) ** 2 * 0.5
    return loss

In [6]:
# Gradient of loss function
def gradient(b_user, b_item, R, irow, jcol):
    b_user_grad = np.zeros(b_user.shape)
    b_item_grad = np.zeros(b_item.shape)
    for i, j in zip(irow, jcol):
        if np.isnan(R[i, j]):
            continue
        b_user_grad[i] += (R[i, j] - b_user[i] - b_item[j])
        b_item_grad[j] += (R[i, j] - b_user[i] - b_item[j])

    return b_user_grad, b_item_grad

In [7]:
# Gradient descent function
def gradient_descent(R, b_user, b_item, irow, jcol, lr, epochs):
    for epoch in range(epochs):
        b_user_grad, b_item_grad = gradient(b_user, b_item, R, irow, jcol)
        prev_b_user = b_user.copy()
        prev_b_item = b_item.copy()
        b_user += lr * b_user_grad 
        b_item += lr * b_item_grad
        if epoch % 10 == 0:
            print('loss:', loss(b_user, b_item, R, irow, jcol))
        # early stopping
        if np.linalg.norm(b_user - prev_b_user) < 1e-2 and np.linalg.norm(b_item - prev_b_item) < 1e-2:
            break

    return b_user, b_item

In [8]:
# Split data into training, validation and test sets
R_copy, test_irow, test_jcol, val_irow, val_jcol, R_test_mask = split_data(R,irow,jcol)

In [9]:
# Initialize biases
b_user = np.random.randn(943)
b_item = np.random.randn(1682)

In [10]:
# Gradient descent
b_user, b_item = gradient_descent(R_copy, b_user, b_item, irow, jcol, 0.001, 1000)

loss: 386185.7983462831
loss: 76323.74909153167
loss: 57531.25698836898
loss: 50753.165402464634
loss: 47381.760636529274
loss: 45438.80671651018
loss: 44214.68400462271
loss: 43394.20103475877
loss: 42818.118457659
loss: 42398.5033710425
loss: 42083.487557364904
loss: 41840.87778573936
loss: 41649.86711670549
loss: 41496.55544779305
loss: 41371.39534164504
loss: 41267.67041990933
loss: 41180.555481552175
loss: 41106.518080937836
loss: 41042.92755682957
loss: 40987.79382658571
loss: 40939.58940786056
loss: 40897.125994679125
loss: 40859.467487061265
loss: 40825.867801859065
loss: 40795.725794759324
loss: 40768.55216746724
loss: 40743.94488148311
loss: 40721.57068440759
loss: 40701.15107972306
loss: 40682.45156223512
loss: 40665.273278620225
loss: 40649.44650677464
loss: 40634.82551219866
loss: 40621.28445638629
loss: 40608.7141159579
loss: 40597.01923178159
loss: 40586.11635157029
loss: 40575.93206199765
loss: 40566.401530538635
loss: 40557.46729532891
loss: 40549.078254978376
loss: 40

In [11]:
# Calculate RMSE
def rmse(R, b_user, b_item, irow, jcol):
    return np.sqrt(2 * loss(b_user, b_item, R, irow, jcol) / len(irow))

In [12]:
# Calculate RMSE on test set
print('Non regularized loss function model RMSE:' , rmse(R, b_user, b_item, test_irow, test_jcol))

Non regularized loss function model RMSE: 0.9654828549684209


## Regularized Loss Function Steps

In [13]:
# Regularized loss function
def loss_reg(b_user, b_item, R, irow, jcol, lmbda):
    loss = 0
    for i, j in zip(irow, jcol):
        if np.isnan(R[i, j]):
            continue
        loss += (R[i, j] - b_user[i] - b_item[j]) ** 2 * 0.5 + lmbda/2 * (b_user[i] ** 2 + b_item[j] ** 2)
    return loss

In [14]:
# Regularized gradient of loss function
def gradient_reg(b_user, b_item, R, irow, jcol, lmbda):
    b_user_grad = np.zeros(b_user.shape)
    b_item_grad = np.zeros(b_item.shape)
    for i, j in zip(irow, jcol):
        if np.isnan(R[i, j]):
            continue
        b_user_grad[i] += (R[i, j] - b_user[i] - b_item[j]) - lmbda * b_user[i]
        b_item_grad[j] += (R[i, j] - b_user[i] - b_item[j]) - lmbda * b_item[j]

    return b_user_grad, b_item_grad

In [15]:
# Regularized gradient descent function
def gradient_descent_reg(R, b_user, b_item, irow, jcol, lmbda, lr, epochs):
    for epoch in range(epochs):
        b_user_grad, b_item_grad = gradient_reg(b_user, b_item, R, irow, jcol, lmbda)
        prev_b_user = b_user.copy()
        prev_b_item = b_item.copy()
        b_user += lr * b_user_grad
        b_item += lr * b_item_grad
        if epoch % 10 == 0:
            print('loss:', loss_reg(b_user, b_item, R, irow, jcol, lmbda))
        # early stopping
        if np.linalg.norm(b_user - prev_b_user) < 1e-2 and np.linalg.norm(b_item - prev_b_item) < 1e-2:
            break

    return b_user, b_item

In [16]:
# Initialize biases
b_user = np.random.randn(943)
b_item = np.random.randn(1682)

In [17]:
# Hyperparameter optimization to find best lambda on validation set
best_rmse = float('inf')
best_lmbda = 0
for lmbda in [0.001, 0.01, 0.1, 1]:
    b_user, b_item = gradient_descent_reg(R_copy, b_user, b_item, irow, jcol, lmbda, 0.0005, 250)
    rmse_val = rmse(R, b_user, b_item, val_irow, val_jcol)
    if rmse_val < best_rmse:
        best_rmse = rmse_val
        best_lmbda = lmbda

loss: 524891.4691990158
loss: 116328.7734424634
loss: 78974.79279840465
loss: 65366.79033974722
loss: 58259.29113117337
loss: 53927.31553904509
loss: 51046.72819013959
loss: 49020.41228135431
loss: 47536.96592583192
loss: 46417.52809592836
loss: 45552.151756448526
loss: 44869.765172124455
loss: 44322.59370498762
loss: 43877.49129199245
loss: 43510.847909074415
loss: 43205.46591621918
loss: 42948.574904866386
loss: 42730.5316712571
loss: 42543.94648343266
loss: 42383.08231279436
loss: 42243.4333042503
loss: 42121.423607805846
loss: 42014.18868008136
loss: 41919.414133293176
loss: 41835.215411857855
loss: 44660.20140206312
loss: 44586.95367466128
loss: 44527.76434630795
loss: 44474.52885727403
loss: 44426.25691665524
loss: 44382.2946569344
loss: 44342.1133573763
loss: 44305.26861319818
loss: 44271.38162235332
loss: 44240.12702078642
loss: 44211.22381008666
loss: 44184.428202465635
loss: 44159.52783143126
loss: 44136.337002597764
loss: 44114.69276375388
loss: 44094.45163227743
loss: 44075

In [18]:
best_lmbda

0.01

In [19]:
# Regularized gradient descent with best lambda
b_user, b_item = gradient_descent_reg(R_test_mask, b_user, b_item, irow, jcol, best_lmbda, 0.0005, 1000)

loss: 93096.79445230114
loss: 50714.63749871168
loss: 47737.6028197279
loss: 46673.455808257044
loss: 46104.61484935234
loss: 45749.550480934246
loss: 45508.61291633486
loss: 45336.21472594272
loss: 45208.151096547525
loss: 45110.28148102286
loss: 45033.780836569334
loss: 44972.8659066574
loss: 44923.59955015559
loss: 44883.217218870755
loss: 44849.72707403505
loss: 44821.66237485197
loss: 44797.922707181846
loss: 44777.66909210223
loss: 44760.2528623619
loss: 44745.16631535431
loss: 44732.00776960806
loss: 44720.45636670428
loss: 44710.25360334301
loss: 44701.18959805646
loss: 44693.09274512882
loss: 44685.82182884627
loss: 44679.25994964616
loss: 44673.30980139378
loss: 44667.8899676672
loss: 44662.93199452516
loss: 44658.37806051201
loss: 44654.179109969475
loss: 44650.29334855906
loss: 44646.68502400737
loss: 44643.32343292327
loss: 44640.182107883564
loss: 44637.23814908124
loss: 44634.47167251156
loss: 44631.86535254679
loss: 44629.40404133409
loss: 44627.07445095442


In [20]:
# Function for regularized RMSE on test set with best model
def rmse_reg(R, b_user, b_item, irow, jcol, lmbda):
    return np.sqrt(2 * loss_reg(b_user, b_item, R, irow, jcol, lmbda) / len(irow))

In [21]:
# Calculate RMSE on test set
print('Regularized RMSE:' , rmse_reg(R, b_user, b_item, test_irow, test_jcol, best_lmbda))

Regularized RMSE: 1.005215290741836
